In [ ]:
import pandas as pd
import numpy as np
import re
from mapping_WM import *


In [ ]:
torob = pd.read_csv('WM_models_data_torob.csv')
digikala = pd.read_csv('WM_models_data_digikala.csv')

In [ ]:
digikala.columns

In [ ]:
# a = torob.columns
# with open('columns.txt', 'w', encoding='utf-8') as f:
#     for i in a:
#         f.write(i + '\n')


In [ ]:
import pandas as pd
import pandas as pd

def map_and_merge_columns(df, column_mappings):
    df = df.copy()  # Avoid modifying the original DataFrame

    for new_col, old_cols in column_mappings.items():
        matching_cols = [col for col in old_cols if col in df.columns]
        if matching_cols:
            if len(matching_cols) > 1:
                df[new_col] = df[matching_cols].apply(
                    lambda row: next((x for x in row if pd.notna(x) and x != ''), None), axis=1
                )
                df.drop(columns=matching_cols, inplace=True)
            else:
                df.rename(columns={matching_cols[0]: new_col}, inplace=True)

    # Select only existing columns
    existing_cols = [col for col in column_mappings.keys() if col in df.columns]
    return df[existing_cols]

digikala_n = map_and_merge_columns(digikala, digikala_mapping)
torob_n = map_and_merge_columns(torob, torob_mapping)


In [ ]:
torob_n['model_code']= torob['model_code']

In [ ]:
digikala_n.shape, torob_n.shape

In [ ]:
torob_n

In [ ]:
wm_concate = pd.concat([digikala_n, torob_n], ignore_index=True)

# Group by 'model_code' and fill missing values with available ones
wm = wm_concate.groupby("model_code", as_index=False).first()

In [ ]:
wm

In [ ]:
print(wm[wm.duplicated(subset='model_code', keep=False)])


In [ ]:
threshold = 0.6
wm = wm.loc[:, wm.isnull().mean() < threshold]

In [ ]:
wm.shape

data cleaning

In [ ]:
def convert_persian_to_english(text):
    persian_digits = "۰۱۲۳۴۵۶۷۸۹"
    english_digits = "0123456789"
    trans_table = str.maketrans(persian_digits, english_digits)
    return text.translate(trans_table)
# Apply the conversion function to the entire DataFrame
wm = wm.applymap(lambda x: convert_persian_to_english(str(x)) if isinstance(x, str) else x)


In [ ]:
cc = [
   "رتبه انرژی",
        "رتبه مصرف انرژی",
        "برچسب مصرف انرژی",
        "گرید انرژی",
        "کلاس مصرف انرژی",
        "نمودار مصرف انرژی",
        "نمودار مصرفی انرژی",
        "مصرف انرژی",
        "میزان مصرف برق",
]
for c in cc:
    print(torob[c].value_counts())
    print()

In [ ]:
wm.energy_label.value_counts()

In [ ]:
wm

In [ ]:
enery_rating_mapping = {
    "+++A": 3,
    "++A": 2,
    "A++": 2,
    "A+": 1,
    "َA++": 2,
    "A+++": 3,
    "A": 0,
    "+A": 1,
    'A+++ -35%': 3,
    '196':np.nan,
    'A+++ -55%': 3,
    'A+++ – 30%': 3,
    'A+++(-49)': 3,
    'A+++ – 10%': 3,
    'کم':np.nan,
    'حداکثر توان: 2000 وات':np.nan,
    '8+++': 3,
    '+++A, ,, تا 30٪ صرفه جویی': 3,
}

In [ ]:
wm['energy_label'] = wm['energy_label'].replace(enery_rating_mapping)
wm['energy_label'].value_counts()

In [ ]:
wm['height'] = [re.findall(r'\d+\.?\d*', str(item))[0] if item is not None else None for item in wm.height]
wm['height'] = wm['height'].astype(float)
wm.loc[wm['height'] > 800, 'height'] = wm.loc[wm['height'] > 800, 'height'] / 10

wm.height.unique()

In [ ]:
wm['depth'] = [re.findall(r'\d+\.?\d*', str(item))[0] if item is not None else None for item in wm.depth]
wm['depth'] = wm['depth'].astype(float)
wm.loc[wm['depth'] > 500, 'depth'] = wm.loc[wm['depth'] > 500, 'depth'] / 10
wm.depth.unique()

In [ ]:
wm['width'] = [re.findall(r'\d+\.?\d*', str(item))[0] if item is not None else None for item in wm.width]
wm['width'] = wm['width'].astype(float)
wm.loc[wm['width'] > 500, 'width'] = wm.loc[wm['width'] > 500, 'width'] / 10
wm.width.unique()

In [ ]:
wm.columns

In [ ]:
wm.drop(columns=['noise_level','display_status'], inplace=True)

In [ ]:
safety_system_mapping = {
    'قفل کودک': 1,
    'سیستم خاموشی خودکار': 1,
    'عیب یابی خودکار': 1,
    'عیب\u200cیابی خودکار':1,
    'yes': 1,
    'دارد': 1,
    'دکمه توقف عملیات':1,
    'سیستم قفل ایمنی':1,
    'ندارد': 0,
    'None': 0,
    None: 0,
}

wm['safety_system'] = wm['safety_system'].replace(safety_system_mapping)

In [ ]:
wm['motor_speed'] = [re.findall(r'\d+\.?\d*', str(item))[0] if item is not None else None for item in wm.motor_speed]
wm['motor_speed'] = wm['motor_speed'].astype(float)
wm.motor_speed.unique() 

In [ ]:
wm['drum_capacity'] = [re.findall(r'\d+\.?\d*', str(item))[0] if item is not None else None for item in wm.drum_capacity]
wm['drum_capacity'] = wm['drum_capacity'].astype(float)
wm['drum_capacity'][wm['drum_capacity'].isin([63])] = 9
wm.drum_capacity.unique()


In [ ]:
door_opening_mapping = {
    
    'به سمت چپ': 'left',
    None:np.nan,
    'از سمت راست':'left',
    'به سمت پایین':'down',
    'از راست به چپ':'left',
    'به سمت چپ با زاویه 150 درجه':'left',
    'جهت باز شدن به سمت چپ':'left',
    'به سمت راست':'right',
    'از سمت راست به چپ':'left',
    'به سمت بالا':'up',
    'از جلو':'left'
}
wm['door_opening_direction'] = wm['door_opening_direction'].replace(door_opening_mapping)

In [ ]:
wm.motor_type.unique()

In [ ]:
motor_mapping = {
    None: "Unknown",
    "universal": "Universal",
    "یونیورسال": "Universal",
    "موتور یونیورسال": "Universal",
    "تسمه‌ای": "Belt Drive",
    "تسمه ای": "Belt Drive",
    "بدون تسمه": "Beltless",
    "نیمه گیربکسی": "Semi-Gearbox",
    "گیربکسی": "Gearbox",
    "دایرکت درایو": "Direct Drive",
    "موتور دایرکت درایو (بدون تسمه)": "Direct Drive",
    "(Direct Drive)دایرکت درایو": "Direct Drive",
    "دایرکت درایو (Direct Drive)": "Direct Drive",
    "دایرکت داریور": "Direct Drive",
    "دایرکت درایو(گیربکسی)": "Direct Drive Gearbox",
    "موتور پیشرفته با تکنولوژیDirect Drive Inverter": "Direct Drive Inverter",
    "اسمارت دایرکت درایو": "Smart Direct Drive",
    "اینورتر BLDC": "BLDC Inverter",
    "BLDC": "BLDC",
    "bldc": "BLDC",
    "BLDC Inverter": "BLDC Inverter",
    "موتور BLDC": "BLDC",
    "BLDC, تسمه ای": "BLDC Belt Drive",
    "BLDC اینورتر": "BLDC Inverter",
    "تسمه‌ای , BLDC": "BLDC Belt Drive",
    "تسمه ای اینورتر (BLDC)": "BLDC Belt Drive",
    "اینورتر (Inverter)": "Inverter",
    "اینورتر": "Inverter",
    "Inverter": "Inverter",
    "اینورتر دیجیتال (Digital Inverter)": "Digital Inverter",
    "دیجیتال اینورتر": "Digital Inverter",
    "اینورتر Q-BLDC Drive": "Q-BLDC Drive",
    "Q-BLDC Drive": "Q-BLDC Drive",
    "اکو سایلنس درایو (EcoSilence Drive)": "EcoSilence Drive",
    "EcoSilence Drive": "EcoSilence Drive",
    "EcoSilence Drive : موتور پیشرفته و فوق العاده بی صدا با عمر طولانی مدت": "EcoSilence Drive",
    "EcoSilence Drive: موتور فوق العاده ساکت با عمر طولانی": "EcoSilence Drive",
    "موتور قدرتمند، کم مصرف و کم صدای EcoSilenceDrive": "EcoSilence Drive",
    "EcoSilence Drive موتور قدرتمند، کم صدا با عمر طولانی": "EcoSilence Drive",
    "موتور فوق العاده ساکت با عمر طولانی": "EcoSilence Drive",
    "اکو سایلنس درایو (EcoSilence Drive) موتور فوق العاده ساکت با عمر طولانی": "EcoSilence Drive",
    "EcoSilence Drive موتور فوق العاده ساکت با عمر طولانی": "EcoSilence Drive",
    "BLDC: موتور فوق العاده ساکت با عمر طولانی": "BLDC",
    "موتور تسمه ای یونیورسال": "Universal Belt Drive",
    "Direct Drive Inverter": "Direct Drive Inverter"
}
wm['motor_type'] = wm['motor_type'].replace(motor_mapping)

In [ ]:
wm.tank_type.unique()

In [ ]:
drum_mapping = {
    None: "Unknown",
    "درب از جلو": "Front Load",
    "در از جلو": "Front Load",
    "درب از بالا": "Top Load",
    "استیل ضد زنگ": "Stainless Steel",
    "استیل": "Stainless Steel",
    "دیگ لانه زنبوری با تکنولوژی Water Magic Cube": "Honeycomb Drum (Water Magic Cube)",
    "دیگ الماسه": "Diamond Drum",
    "درام الماسه": "Diamond Drum",
    "الماسه": "Diamond Drum",
    "الماسه حباب ساز": "Bubble Diamond Drum",
    "دیگ زمردی": "Emerald Drum",
    "(Emerald) درام زمردی": "Emerald Drum",
    "درام کلاستر (Cluster)": "Cluster Drum",
    "کریستالی (Honeycomb)": "Crystal Honeycomb Drum"
}
wm['tank_type'] = wm['tank_type'].replace(drum_mapping)

In [ ]:
wm['price'] = wm['price'].str.replace('تومان', '').str.replace('٫', '').str.replace(',', '').str.strip()
wm['price'] = wm['price'].replace('ناموجود', np.nan)  # Replace 'ناموجود' with NaN
wm['price'] = pd.to_numeric(wm['price'], errors='coerce')  # Convert to numeric, coercing errors to NaN

print(wm['price'].unique())

In [ ]:
wm['brand'] = wm['model_code'].str.split('-').str[0]
wm['brand'].value_counts()

In [ ]:
wm

In [ ]:
wm.isna().sum()

filling nulls

In [ ]:
col = [
    'safety_system',
    'motor_speed',
    'door_opening_direction',
    'energy_label'
]
for c in col:
    wm[c] = wm[c].fillna(wm[c].mode()[0])
# wm['energy_label'] = wm['energy_label'].fillna(wm['energy_label'].mode()[0])

In [ ]:
wm['price'] = wm.groupby(['brand'])['price'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
wm.dropna(subset=['height','depth','width','drum_capacity'], how='all' , inplace=True)
wm.shape

In [ ]:
# for col in ['height', 'depth', 'width','drum_capacity']:
#     wm[col] = wm.groupby(['brand'])[col].transform(lambda x: x.fillna(x.mean()))

for col in ['height', 'depth', 'width', 'drum_capacity']:
    wm[col] = wm.groupby('brand')[col].transform(lambda x: x.fillna(round(x.mean(), 1)))
        # wm[col] = wm.groupby(['brand'])[col].transform(lambda x: x.fillna(round(x.mean())))

In [ ]:
wm[['height','depth','width','drum_capacity']] 

In [ ]:
wm.dropna(inplace=True)
wm.shape

clustering

In [ ]:
dw_cp = wm.copy()
dw_cp.drop(['model_code'], axis=1, inplace=True)
categorical_cols = dw_cp.select_dtypes(include=['object', 'category']).columns.tolist()
dw_cp.shape

In [ ]:
dw_cp = pd.get_dummies(dw_cp, columns=categorical_cols)

In [ ]:
dw_cp.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute the correlation matrix
corr_matrix = dw_cp.corr()

# Plot the heatmap

plt.figure(figsize=(20, 20))
sns.heatmap(corr_matrix,annot=False, cmap='coolwarm', linewidths=0.2)

# Title
plt.title("Correlation Heatmap of washing machine features")

# Show plot
plt.show()


In [ ]:
threshold = 0.8
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns 
          if any((upper_tri[column] > threshold) | (upper_tri[column] < -threshold))]

# Drop the highly correlated columns
wm_cleaned = dw_cp.drop(columns=[col for col in to_drop if col in dw_cp.columns])

print(f"Dropped columns: {to_drop}")
# print(f"Original shape: {wm.shape}")
print(f"New shape: {wm_cleaned.shape}")

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Assuming df is your dataframe
scaler =StandardScaler()
df_scaled = scaler.fit_transform(wm_cleaned)


finding number of cluster for kmeans with elbow method

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(df_scaled)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, 11), inertia)
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score
kmeans = KMeans(n_clusters=3, random_state=42)
wm['cluster'] = kmeans.fit_predict(df_scaled)
# internal Evaluation Metrics
score = silhouette_score(df_scaled, wm['cluster'])  # X = dataset, labels = cluster assignments
print(f"Silhouette Score: {score:.3f}") #Closer to 1 → Well-defined clusters

dbi = davies_bouldin_score(df_scaled, wm['cluster'])
print(f"Davies-Bouldin Index: {dbi:.3f}") #Lower values indicate better clustering.

ch = calinski_harabasz_score(df_scaled, wm['cluster'])
print(f"Calinski-Harabasz Index: {ch:.3f}") #Higher values indicate better clustering.

# Add the cluster labels to the original dataframe
print(wm[['model_code', 'cluster']].head())


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

# Reduce to 3D for visualization
pca = PCA(n_components=2,svd_solver='full')
df_pca = pca.fit_transform(df_scaled)

# Create a DataFrame for the reduced features and cluster labels
df_pca_df = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
df_pca_df['cluster'] = wm['cluster']

# 3D Scatter Plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111)

# Scatter plot with cluster colors
scatter = ax.scatter(df_pca_df['PC1'], df_pca_df['PC2'], 
                     c=df_pca_df['cluster'], cmap='viridis', alpha=0.8)

# Labels and title
ax.set_title('3D PCA of Clusters')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

# Add color bar
plt.colorbar(scatter, ax=ax, shrink=0.5, aspect=5)

# Show plot
plt.show()


<h1>DBSCAN</h1>

finding min_sample

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import matplotlib.pyplot as plt

for k in [5,10,15,20]:  # Test different min_samples values
    nbrs = NearestNeighbors(n_neighbors=k).fit(df_scaled)
    distances, indices = nbrs.kneighbors(df_scaled)
    
    sorted_distances = np.sort(distances[:, k-1])  # k-th nearest neighbor
    plt.plot(sorted_distances, label=f'k={k}')

plt.xlabel("Data Points Sorted")
plt.ylabel("k-Nearest Neighbor Distance")
plt.title("KNN Distance Plot for Different min_samples")
plt.legend()
plt.grid()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from mpl_toolkits.mplot3d import Axes3D

# Apply DBSCAN
dbscan = DBSCAN(eps=5 ,min_samples=5)  # Adjust eps & min_samples as needed
wm['cluster_dbscan'] = dbscan.fit_predict(df_scaled)  # Assign cluster labels

score = silhouette_score(df_scaled, wm['cluster_dbscan'])  # X = dataset, labels = cluster assignments
print(f"Silhouette Score: {score:.3f}") #Closer to 1 → Well-defined clusters

dbi = davies_bouldin_score(df_scaled, wm['cluster_dbscan'])
print(f"Davies-Bouldin Index: {dbi:.3f}") #Lower values indicate better clustering.

ch = calinski_harabasz_score(df_scaled, wm['cluster_dbscan'])
print(f"Calinski-Harabasz Index: {ch:.3f}") #Higher values indicate better clustering.


In [ ]:
# sbs['best_dbscan']=best_labels

In [ ]:
# Reduce to 3D for visualization using PCA
pca = PCA(n_components=3)
df_pca = pca.fit_transform(df_scaled)

# Create a DataFrame for visualization
df_pca_df = pd.DataFrame(df_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca_df['cluster'] = wm['cluster_dbscan']  # Add DBSCAN cluster labels

# Plot 3D Scatter
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot points, coloring them by cluster
scatter = ax.scatter(df_pca_df['PC1'], df_pca_df['PC2'], df_pca_df['PC3'], 
                     c=df_pca_df['cluster'], cmap='viridis', alpha=0.8)

# Labels and title
ax.set_title('3D PCA of DBSCAN Clusters')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')

# Add color bar
plt.colorbar(scatter, ax=ax, shrink=0.5, aspect=5)

# Show plot
plt.show()


In [ ]:
wm.cluster_dbscan.value_counts()

In [ ]:
from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score

# for k in range(2, 10):  # Try different k values
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     labels = kmeans.fit_predict(df_scaled)
#     score = silhouette_score(df_scaled, labels)
#     print(f'k={k}, Silhouette Score={score}')


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs


In [ ]:
bic_scores = []
aic_scores = []
n_components_range = range(1, 10)

for n in n_components_range:
    gmm = GaussianMixture(n_components=n, covariance_type='spherical', random_state=42)
    gmm.fit(df_scaled)
    bic_scores.append(gmm.bic(df_scaled))
    aic_scores.append(gmm.aic(df_scaled))

# Plot BIC and AIC scores
plt.figure(figsize=(8, 4))
plt.plot(n_components_range, bic_scores, label='BIC', marker='o')
plt.plot(n_components_range, aic_scores, label='AIC', marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.legend()
plt.title('Optimal Number of Clusters using BIC & AIC')
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score
# Fit GMM model
gmm = GaussianMixture(n_components=8, covariance_type='tied', random_state=42)
gmm.fit(df_scaled)

# Predict cluster labels
clusters = gmm.predict(df_scaled)

# Silhouette Score (Higher is better)
sil_score = silhouette_score(df_scaled, clusters)
print(f'Silhouette Score: {sil_score:.4f}')
dbi = davies_bouldin_score(df_scaled, clusters)
print(f"Davies-Bouldin Index: {dbi:.3f}") #Lower values indicate better clustering.

ch = calinski_harabasz_score(df_scaled, clusters)
print(f"Calinski-Harabasz Index: {ch:.3f}") #Higher values indicate better clustering.

# Plot clustered data
plt.scatter(df_scaled[:, 0], df_scaled[:, 1], c=clusters, cmap='viridis', s=15)
plt.scatter(gmm.means_[:, 0], gmm.means_[:, 1], c='red', marker='x', s=100, label='Cluster Centers')
plt.title("GMM Clustering Results")
plt.legend()
plt.show()


In [ ]:
# Reduce to 3D for visualization using PCA
pca = PCA(n_components=3)
df_pca = pca.fit_transform(df_scaled)

# Create a DataFrame for visualization
df_pca_df = pd.DataFrame(df_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca_df['cluster'] = clusters # Add GMM cluster labels

# Plot 3D Scatter
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot points, coloring them by cluster
scatter = ax.scatter(df_pca_df['PC1'], df_pca_df['PC2'], df_pca_df['PC3'], 
                     c=df_pca_df['cluster'], cmap='viridis', alpha=0.8)

# Labels and title
ax.set_title('3D PCA of DBSCAN Clusters')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')

# Add color bar
plt.colorbar(scatter, ax=ax, shrink=0.5, aspect=5)

# Show plot
plt.show()


In [ ]:
# Get probability of each point belonging to clusters
probs = gmm.predict_proba(df_scaled)
print("Cluster Probabilities (First 5 points):\n", probs[:50])


In [ ]:
df_scaled


In [ ]:
df_scaled.shape

In [ ]:
from itertools import combinations
best_score = -1
feature_indices = list(range(47))  # 11 features
# Try all combinations of 5 features out of 11
for feature_subset in combinations(feature_indices, 5):
    X_subset = df_scaled[:, feature_subset]  # Select features
    kmeans = KMeans(n_clusters=5, random_state=42).fit(X_subset)  # Assume 3 clusters
    score = silhouette_score(X_subset, kmeans.labels_)  # Measure clustering quality
    
    if score > best_score:
        best_score = score
        best_features = feature_subset

print("Best Feature Indices:", best_features)
print("Best Silhouette Score:", best_score)

In [ ]:
dw_cp

In [ ]:
best_feature_names = [dw_cp.columns[i] for i in best_features]
best_feature_names

In [ ]:
# Reduce to 3D for visualization using PCA
pca = PCA(n_components=3)
df_pca = pca.fit_transform(df_scaled)

# Create a DataFrame for visualization
df_pca_df = pd.DataFrame(df_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca_df['cluster'] = kmeans.labels_ # Add GMM cluster labels

# Plot 3D Scatter
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot points, coloring them by cluster
scatter = ax.scatter(df_pca_df['PC1'], df_pca_df['PC2'], df_pca_df['PC3'], 
                     c=df_pca_df['cluster'], cmap='viridis', alpha=0.8)

# Labels and title
ax.set_title('3D PCA of DBSCAN Clusters')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')

# Add color bar
plt.colorbar(scatter, ax=ax, shrink=0.5, aspect=5)

# Show plot
plt.show()
